## 初期設定

In [1]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
import src.config as cfg
import src.data_loader as dl
import src.cleaning_utils as cu
print("🔁 autoreload 有効化完了")
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

🔁 autoreload 有効化完了


In [2]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

{'project_name': 'stock_screening',
 'data_path': {'raw': './data/raw',
  'interim': './data/interim',
  'processed': './data/processed'},
 'files': {'raw': ['fy-balance-sheet.csv',
   'fy-cash-flow-statement.csv',
   'fy-profit-and-loss.csv',
   'fy-stock-dividend.csv'],
  'reference': ['CodeData.csv'],
  'interim': None,
  'processed': None},
 'years': [2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021,
  2022,
  2023,
  2024,
  2025],
 'na_values': ['-', '0', 'NaN', ''],
 'output': {'base_path': './output'}}

## 本ファイルの説明
データが最低限の品質基準を満たしているかを網羅的に確認します。


### データのロードの確認
まずは欠損値の表現をチェックします。  
その後、各ファイルの企業コード数、年度数、列数をチェックします。

<details>
<summary><b>結果</b></summary>

2025年の企業コード数と年度数が多くなっています。  
2025年には2023年と2024年の更新データが含まれていると推測されます。  
そのため、2023年と2024年のデータを最新に更新し、2025年のデータから抜き取ります
</details>

In [ ]:
# 欠損値の表現確認
print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"],[""]
)

print("")
print("結合します。ファイル名と会計年の列を追加しておきます")
processed_dfs = []
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    df_copy = df.copy()
    df_copy["ファイル名"] = filename
    df_copy["会計年"] = year
    processed_dfs.append(df_copy)
merged_df = pd.concat(processed_dfs, ignore_index=True)
final_df = merged_df.groupby(["コード", "年度"]).first()
final_df = final_df.reset_index()
display(final_df)

print("")
print("欠損値の値を探索します。この結果はsetting.yamlに登録します")




ファイル毎にデータを取得します

結合します


,コード,年度,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率,...,純利益,EPS,ROE,ROA,一株配当,剰余金の配当,自社株買い,配当性向,総還元性向,純資産配当率
0,1301,2010-03-01,64301000000,18538000000,18391000000,12589000000,26724000000,2712000000,1651.87,28.1,...,1086000000,99.38,6.02,1.69,50,529000000,0,48.71,48.7,-
1,1301,2011-03-01,76925000000,17555000000,17785000000,12119000000,36102000000,3622000000,1562.75,22.2,...,58000000,5.31,0.34,0.08,50,529000000,135000000,912.07,1144.8,-
2,1301,2012-03-01,84937000000,17212000000,17683000000,12017000000,39205000000,2797000000,1552.5,20,...,423000000,38.71,2.49,0.5,50,525000000,0,124.11,124.1,-
3,1301,2013-03-01,83245000000,18683000000,18512000000,12846000000,23191000000,8153000000,1687.1,22.1,...,1269000000,116.12,6.88,1.52,50,525000000,0,41.4,41.4,3
4,1301,2014-03-01,84319000000,19930000000,20954000000,15289000000,19734000000,11997000000,1875.72,23.4,...,2968000000,282.58,15.07,3.52,50,525000000,-,17.7,17.7,2.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62240,9997,2021-03-01,240211000000,113231000000,115752000000,94667000000,5909000000,56797000000,1164.96,46.9,...,11036000000,114.17,9.8,4.59,16.5,1546000000,-,14.5,14.5,1.5
62241,9997,2022-03-01,254178000000,119044000000,124228000000,103131000000,12909000000,60642000000,1223.24,46.5,...,10204000000,105.54,8.63,4.01,19,1740000000,-,18,18,1.6
62242,9997,2023-03-01,285592000000,126436000000,129765000000,108663000000,20326000000,85462000000,1297.93,43.9,...,7417000000,76.71,5.91,2.6,20,1885000000,-,26.1,26.1,1.6
62243,9997,2024-03-01,300691000000,136182000000,133652000000,112545000000,13090000000,98563000000,1398.06,45.0,...,5839000000,60.39,4.32,1.94,20.5,1958000000,-,33.9,33.9,1.5


In [ ]:
# 財務データ 結合前
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 60)

def get_file_info(df_datas_by_all_files):
    df_file_info = pd.DataFrame(columns=["file","year","code_counts","year_counts","column_counts"])
    for file, year in df_datas_by_all_files.keys():
        df = df_datas_by_all_files[(file,year)]
        # len(df_file_info)を次の新しい行のインデックスとして明示的に指定します。
        df_file_info.loc[len(df_file_info)] = [
            file, year, df["コード"].nunique(), df["年度"].nunique(),df.columns.nunique()
        ]
    return df_file_info

print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"],["-","","0"]
)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("企業コード数、エンドの数、列数をファイルごとにまとめます。")
df_file_info = get_file_info(df_DATAs_BY_ALL_FILEs)
#display(df_file_info)

print("")
print("各ファイルの年推移を可視化します。")
"""fig = px.line(df_file_info,x="year",y="code_counts",color="file")
fig.show()
fig = px.line(df_file_info,x="year",y="year_counts",color="file")
fig.show()
fig = px.line(df_file_info,x="year",y="column_counts",color="file")
fig.show()"""

print("")
print("列数はOKです")
print("最新年のファイルに登録されている年度を表示します。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df["年度"].unique())

print("")
print("最新年のファイルに過去のデータがあります。")
print("最新年に登録されている過去の年度と過去のデータが同じか確認します")
print("まずは、年度を限定して、コードの一覧を見ます")
#display(df[df["年度"]=="2024/12"])

print("")
print("コードを限定し同じかどうか調べます。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("違っています。これは更新データと思われます。")
print("最新年度にある過去のデータを取得し、過去のデータを更新します")
df_DATAs_BY_ALL_FILEs = dl.update_duplicated(df_DATAs_BY_ALL_FILEs, 2025)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("正しく処理が行われ、最新年に登録されている過去の年度と過去のデータが同じか確認します。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("最後に最新年度にある過去のデータを消去します。")
cutoff_date = pd.to_datetime("2025-01-01")
for file, year in df_DATAs_BY_ALL_FILEs.keys():
    if year == 2025:
        df = df_DATAs_BY_ALL_FILEs[(file,year)]
        df = df[df["年度"] >= cutoff_date]
        df_DATAs_BY_ALL_FILEs[(file,year)] = df
print("")
print("各ファイルの年推移をもう一度可視化し、改善していることを確かめます。") 
df_file_info_after = get_file_info(df_DATAs_BY_ALL_FILEs)
df_file_info_after["区分"] = "処理後"
df_file_info_after = df_file_info_after.set_index(["file", "year"]).sort_index()
df_file_info["区分"] = "処理前"
df_file_info = df_file_info.set_index(["file", "year"]).sort_index()
df_compare = pd.concat([df_file_info,df_file_info_after]).sort_index().reset_index()
#display(df_compare)
fig = px.line(df_compare,x="year",y="code_counts",color="区分")
fig.show()


In [ ]:
# 財務データ ファイルごとの結合
df_DATAs_BY_FILEs = dl.load_yearly_data(settings["data_path"], settings["years"], settings["files"],)
target_file = settings["files"][0]
display(df_DATAs_BY_FILEs[target_file][df_DATAs_BY_FILEs[target_file]["コード"]=="130A"])

In [ ]:
# 財務データ　全結合
df_ALL_DATAs = dl.merge_all_data(df_DATAs_BY_FILEs)
print(df_ALL_DATAs.shape)
#display(df_ALL_DATAs)
display(df_ALL_DATAs["コード"].unique())
display(df_ALL_DATAs["年度"].nunique())

In [ ]:
# 上場企業の情報
df = dl.load_code_list_info(settings["data_path"], settings["files_reference"][0])
print(settings["files_reference"][0], df.shape)
display(df)

### データ型の整合性チェック

### 初期品質の概要把握